In [1]:
import numpy as np
import pandas as pd

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

def mi_split (df):

    X = df.drop(['precio'], axis = 1)
    Y = df['precio']
    x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.35)
    return x_train, x_test, y_train, y_test

# x_train y_train son mi TRAIN
# x_test y_test son mi VALIDACION

from scipy.stats import uniform, randint
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
from sklearn import preprocessing

def mean_target_encoding(train, nombrefeat, nombretarget):
    
    temp = train.groupby(nombrefeat)[nombretarget].transform(np.mean)
    train[nombrefeat + "_mean"]=(temp-temp.min())/(temp.max()-temp.min())
    
   # temp = train.groupby(nombrefeat)[nombretarget].transform(np.mean) #Saco promedio
   # temp = preprocessing.scale(temp) #Normalizo
   # train[nombrefeat + "_mean"] = temp #Dejo en el DF
    
    return train

def mean_target_decoding(x_test, nombrefeat, x_train):
    
    nombrefeatmean = nombrefeat + "_mean"
    
    temp = x_train.loc[:,[nombrefeat,nombrefeatmean]]
    temp = temp.set_index(nombrefeat)
    temp = temp.drop_duplicates()
    temp = temp.T.squeeze()
    values = x_test[nombrefeat].map(temp)
    x_test[nombrefeatmean] = values 

    return x_test

x_test = pd.read_csv("test.csv")


def armar_set(train):
    
    #Puedo resolver de forma general las que son iguales para train y test
    #es decir, aquellas que no filtran informacion a los de validacion
    
    train = train.drop(['descripcion', 'titulo', 'direccion', 'fecha', 'id'], axis = 1)
    
    train['servicios'] = train['piscina'] + train['gimnasio'] + train['usosmultiples']
    
    #Elimino los residuos
    train = train.drop(['piscina', 'gimnasio','usosmultiples','escuelascercanas', 
                        'centroscomercialescercanos'], axis = 1)
    
    #Hago el split                    
    x_train, x_test, y_train, y_test = mi_split(train)
        
    x_train["precio"] = y_train

    
    #Calculo los mean target
    x_train_mean = mean_target_encoding(x_train, "provincia", "precio")
    x_train_mean['precio'] = y_train
    x_train_mean = mean_target_encoding(x_train_mean, "tipodepropiedad", "precio")
    x_train_mean = mean_target_encoding(x_train_mean, "ciudad", "precio")
    
    #Se los asigno a los test (NO LOS CALCULO CON ELLOS!!!!!!)
    x_test = mean_target_decoding(x_test, "provincia", x_train_mean)
    x_test = mean_target_decoding(x_test, "tipodepropiedad", x_train_mean)
    x_test = mean_target_decoding(x_test, "ciudad", x_train_mean)

    backup = x_train_mean
    
    x_train = x_train_mean.drop(["precio","provincia","tipodepropiedad","ciudad"], axis=1)
    x_test.drop(["provincia","tipodepropiedad", "ciudad"], axis=1, inplace = True)

    
    return x_train, x_test, y_train, y_test, backup

x_train, x_test, y_train, y_test, x_train_full = armar_set(train)

In [6]:
import sklearn as sk
import sklearn.neural_network

from sklearn.impute import SimpleImputer

from IPython.core.display import display, HTML


lr = 0.01           # learning rate
nn = [2, 16, 8, 1]  # número de neuronas por capa.

# Creamos el objeto del modelo de red neuronal multicapa.
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp = imp.fit(x_train)

x_train = imp.transform(x_train)

clf = sk.neural_network.MLPRegressor(solver='sgd', 
                                     learning_rate_init=lr, 
                                     hidden_layer_sizes=tuple(nn[1:]),
                                     verbose=True,
                                     n_iter_no_change=1000,
                                     batch_size = 64,
                                     max_iter = 500)


# Y lo entrenamos con nuestro datos.
clf.fit(x_train, y_train)

Iteration 1, loss = 7094092572598320128.00000000
Iteration 2, loss = 7128899699636708352.00000000
Iteration 3, loss = 7123468206480669696.00000000
Iteration 4, loss = 7118040850887426048.00000000
Iteration 5, loss = 7112617630545178624.00000000
Iteration 6, loss = 7107198541881694208.00000000
Iteration 7, loss = 7101783582613612544.00000000
Iteration 8, loss = 7096372748829576192.00000000
Iteration 9, loss = 7090966037340255232.00000000
Iteration 10, loss = 7085563445209064448.00000000
Iteration 11, loss = 7080164969393682432.00000000
Iteration 12, loss = 7074770606831702016.00000000
Iteration 13, loss = 7069380354302403584.00000000
Iteration 14, loss = 7063994207977942016.00000000
Iteration 15, loss = 7058612165802839040.00000000
Iteration 16, loss = 7053234224457266176.00000000
Iteration 17, loss = 7047860379991738368.00000000
Iteration 18, loss = 7042490630322119680.00000000
Iteration 19, loss = 7037124971173235712.00000000
Iteration 20, loss = 7031763400893176832.00000000
Iteration

Iteration 164, loss = 6300836513278106624.00000000
Iteration 165, loss = 6296035919325250560.00000000
Iteration 166, loss = 6291238983149077504.00000000
Iteration 167, loss = 6286445702046186496.00000000
Iteration 168, loss = 6281656072281480192.00000000
Iteration 169, loss = 6276870092141698048.00000000
Iteration 170, loss = 6272087757954905088.00000000
Iteration 171, loss = 6267309068048697344.00000000
Iteration 172, loss = 6262534018519147520.00000000
Iteration 173, loss = 6257762607435634688.00000000
Iteration 174, loss = 6252994831767545856.00000000
Iteration 175, loss = 6248230688181753856.00000000
Iteration 176, loss = 6243470174618780672.00000000
Iteration 177, loss = 6238713288602425344.00000000
Iteration 178, loss = 6233960026131241984.00000000
Iteration 179, loss = 6229210385074777088.00000000
Iteration 180, loss = 6224464363529167872.00000000
Iteration 181, loss = 6219721957046579200.00000000
Iteration 182, loss = 6214983164521675776.00000000
Iteration 183, loss = 621024798

Iteration 325, loss = 5573203928256420864.00000000
Iteration 326, loss = 5568957716228983808.00000000
Iteration 327, loss = 5564714739456623616.00000000
Iteration 328, loss = 5560474995447911424.00000000
Iteration 329, loss = 5556238481787098112.00000000
Iteration 330, loss = 5552005195919637504.00000000
Iteration 331, loss = 5547775135033309184.00000000
Iteration 332, loss = 5543548297572480000.00000000
Iteration 333, loss = 5539324680248062976.00000000
Iteration 334, loss = 5535104280613127168.00000000
Iteration 335, loss = 5530887096680526848.00000000
Iteration 336, loss = 5526673126307103744.00000000
Iteration 337, loss = 5522462365830150144.00000000
Iteration 338, loss = 5518254814003903488.00000000
Iteration 339, loss = 5514050467814913024.00000000
Iteration 340, loss = 5509849324659610624.00000000
Iteration 341, loss = 5505651382651204608.00000000
Iteration 342, loss = 5501456639281938432.00000000
Iteration 343, loss = 5497265091072625664.00000000
Iteration 344, loss = 549307673

Iteration 486, loss = 4929599768544625664.00000000
Iteration 487, loss = 4925843918046291968.00000000
Iteration 488, loss = 4922090927959296000.00000000
Iteration 489, loss = 4918340798167507968.00000000
Iteration 490, loss = 4914593524831245312.00000000
Iteration 491, loss = 4910849107355441152.00000000
Iteration 492, loss = 4907107542402561024.00000000
Iteration 493, loss = 4903368827844619264.00000000
Iteration 494, loss = 4899632962461060096.00000000
Iteration 495, loss = 4895899942693455872.00000000
Iteration 496, loss = 4892169767774169088.00000000
Iteration 497, loss = 4888442434381062144.00000000
Iteration 498, loss = 4884717940951998464.00000000
Iteration 499, loss = 4880996284840058880.00000000
Iteration 500, loss = 4877277465128573952.00000000


/home/giaco/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPRegressor(activation='relu', alpha=0.0001, batch_size=64, beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(16, 8, 1), learning_rate='constant',
             learning_rate_init=0.01, max_iter=500, momentum=0.9,
             n_iter_no_change=1000, nesterovs_momentum=True, power_t=0.5,
             random_state=None, shuffle=True, solver='sgd', tol=0.0001,
             validation_fraction=0.1, verbose=True, warm_start=False)

In [7]:
x_test = imp.transform(x_test)

In [8]:
prediccion = clf.predict(x_test)

In [9]:
print('MAE:', mean_absolute_error(y_test, prediccion))

MAE: 1597524.6606868205
